In [5]:
import copy
import pandas as pd

In [35]:
FuzzyS = {10 : (9,10,10), 9 : (8,9,10), 8 : (7,8,9), 7 : (6,7,8),
          6 : (5,6,7), 5 : (4,5,6), 4 : (3,4,5), 3 : (2,3,4), 
          2 : (1,2,3), 1 : (1,1,2)}
FuzzyO = {10 : (8,9,10,10), 9 : (8,9,10,10), 8 : (6,7,8,9), 
          7 : (6,7,8,9), 6 : (3,4,6,7), 5 : (3,4,6,7), 4 : (3,4,6,7),
          3 : (1,2,3,4), 2 : (1,2,3,4), 1 : (1,1,2)}
FuzzyD = {10 : (9,10,10), 9 : (8,9,10), 8 : (7,8,9), 7 : (6,7,8),
          6 : (5,6,7), 5 : (4,5,6), 4 : (3,4,5), 3 : (2,3,4), 
          2 : (1,2,3), 1 : (1,1,2)}
FuzzyL = {'VL' : (0, 0, 0.25), 'L' : (0, 0.25, 0.5), 'M' : (0.25, 0.5, 0.75),
          'H' : (0.5, 0.75, 1), 'VH' : (0.75, 1, 1)}

varResiko = {
                'E1':'Perencanaan yang tidak tepat akibat perubahan iklim',
                'E2':'Target penanaman yang tidak sesuai',
                'E3':'Kualitas yang tidak sesuai',
                'E4':'Kurangnya tenaga kerja',
                'E5':'Harga pupuk yang fluktuatif',
                'E6':'Ketersediaan air tidak memadai',
                'E7':'Tenaga kerja kurang terampil',
                'E8':'Terdapat hama',
                'E9':'Pemanenan tidak serentak ',
                'E10':'Kualitas buah kopi yang tidak sesuai dengan standar',
                'E11':'Biji kopi kualitas rendah',
                'E12':'Mesin yang digunakan tidak stabil',
                'E13':'Pekerja kesulitan mengoperasikan mesin',
                'E14':'Terbuangnya kopi akibat tidak tersangrai dengan sempurna',
                'E15':'Terdapat kerikil pada biji kopi yang telah disangrai',
                'E16':'Kurang memadainya peralatan ',
                'E17':'Kurang menariknya kemasan yang digunakan',
                'E18':'Kebersihan tempat penyimpanan kurang',
                'E19':'Terjadi keterlambatan pengiriman',
                'E20':'Rendahnya tingkat kepuasan konsumen',
                'E21':'Profit yang dihasilkan tidak stabil',
                'E22':'Pemutusan kerjasama antar pemasok dengan distributor',
             }

class Pakar:

    Severity = 0
    Occurance = 0
    Detection = 0
    LS = ''
    LO = ''
    LD = ''

    def __init__(self, role, weight):
        self.role = role
        self.weight = weight

    def RPN(self):
        return self.Severity*self.Occurance*self.Detection

class Risk:

    def __init__(self, riskCode, riskName, listPakar):
        self.riskCode = riskCode
        self.riskName = riskName
        self.listPakar = listPakar

    def RIS(self):
        weightxS = 0
        for pakar in self.listPakar:
            for number in FuzzyS[pakar.Severity]:
                weightxS = weightxS + pakar.weight*number
        ris = (weightxS/1000)/3
        return round(ris, 2)

    def RIO(self):
        weightxO = 0
        for pakar in self.listPakar:
            for number in FuzzyS[pakar.Occurance]: # FUZZY O/S
                weightxO = weightxO + pakar.weight*number
        rio = (weightxO/1000)/3
        return round(rio, 2)

    def RID(self):
        weightxD = 0
        for pakar in self.listPakar:
            for number in FuzzyD[pakar.Detection]:
                weightxD = weightxD + pakar.weight*number
        rid = (weightxD/1000)/3
        return round(rid, 2)
    
class Kategori:

    def __init__(self, kategoriName, listResiko):
        self.kategoriName = kategoriName
        self.listResiko = listResiko
        self.S = {listResiko[0].listPakar[0].role : '', listResiko[0].listPakar[1].role : '', listResiko[0].listPakar[2].role : ''}
        self.O = {listResiko[0].listPakar[0].role : '', listResiko[0].listPakar[1].role : '', listResiko[0].listPakar[2].role : ''}
        self.D = {listResiko[0].listPakar[0].role : '', listResiko[0].listPakar[1].role : '', listResiko[0].listPakar[2].role : ''}

    def Severity(self):
        weightxS = 0
        for pakar in self.listResiko[0].listPakar:
            for key, value in self.S.items():
                if pakar.role == key and value != '':
                    for number in FuzzyL[value]:
                        weightxS = weightxS + pakar.weight*number
        S = (weightxS/100)/3
        return round(S, 3)
    
    def Occurance(self):
        weightxO = 0
        for pakar in self.listResiko[0].listPakar:
            for key, value in self.O.items():
                if pakar.role == key and value != '':
                    for number in FuzzyL[value]:
                        weightxO = weightxO + pakar.weight*number
        O = (weightxO/100)/3
        return round(O, 3)
    
    def Detection(self):
        weightxD = 0
        for pakar in self.listResiko[0].listPakar:
            for key, value in self.D.items():
                if pakar.role == key and value != '':
                    for number in FuzzyL[value]:
                        weightxD = weightxD + pakar.weight*number
        D = (weightxD/100)/3
        return round(D, 3)
    
    def WS(self):
        return round(self.Severity() + self.Occurance() + self.Detection(), 3)
    
    def MWS(self):
        return round(self.Severity()/self.WS(), 3)
    
    def MWO(self):
        return round(self.Occurance()/self.WS(), 3)
    
    def MWD(self):
        return round(self.Detection()/self.WS(), 3)

In [43]:
# Inisiasi pakar
listPakar = [Pakar('Akademisi', 40), Pakar('UMKM1', 30), Pakar('UMKM2', 30)]

In [44]:
# Inisiasi risk
# riskNum = int(input("Enter the number of risk: ")) # Jumlah dari ISM

listCode = ['E9', 'E11', 'E19']
listResiko = []

for i in range(len(listCode)):
    risk = Risk(listCode[i], varResiko[listCode[i]], copy.deepcopy(listPakar))
    listResiko.append(risk)

In [47]:
# Input SOD
for risk in listResiko:
        for pakar in risk.listPakar:
            pakar.Severity = int(input("Enter severity risk " + str(risk.riskCode)+ " for " + str(pakar.role) + ": "))
            pakar.Occurance = int(input("Enter occurance risk " + str(risk.riskCode)+ " for " + str(pakar.role) + ": "))
            pakar.Detection = int(input("Enter detection risk " + str(risk.riskCode)+ " for " + str(pakar.role) + ": "))

# Input Linguistik
for pakar in listPakar:
        pakar.LS = input("Enter Linguistik Severity for " + str(pakar.role) + ": ")
        pakar.LO = input("Enter Linguistik Occurance for " + str(pakar.role) + ": ")
        pakar.LD = input("Enter Linguistik Detection for " + str(pakar.role) + ": ")

In [ ]:
for risk in listResiko:
    for i in range(len(risk.listPakar)):
        risk.listPakar[i].LS = listPakar[i].LS
        risk.listPakar[i].LO = listPakar[i].LO
        risk.listPakar[i].LD = listPakar[i].LD

In [4]:
# Inisiasi kategori resiko
jumlahKategori = int(input("Enter the number of kategori: "))
listKategori = []

for i in range(jumlahKategori):
    kategoriName = input("Enter kategori "+ str(i+1)+": ")
    
    # Inisiasi pakar
    jumlahPakar = int(input("Enter the number of pakar: "))

    while True:
        listPakar = []
        for i in range(jumlahPakar):
            role = input("Enter role pakar "+ str(i+1)+": ")
            weight = int(input("Enter weight pakar (%)"+ str(i+1)+": "))
            pakar = Pakar(role, weight)
            listPakar.append(pakar)

        jumlahWeight = 0
        for j in listPakar:
            jumlahWeight = jumlahWeight + j.weight

        if jumlahWeight != 100:
            print('Incorrect total weight, please try again')
        else:
            break

    # Inisiasi resiko
    jumlahResiko = int(input("Enter the number of resiko: "))
    listResiko = []

    for i in range(jumlahResiko):
        riskName = input("Enter resiko "+ str(i+1)+": ")
        risk = Risk(riskName, copy.deepcopy(listPakar))
        listResiko.append(risk)


    kategori = Kategori(kategoriName, listResiko)
    listKategori.append(kategori)

ValueError: invalid literal for int() with base 10: ''

In [4]:
# Input SOD
for kategori in listKategori:
    for risk in kategori.listResiko:
        for pakar in risk.listPakar:
            pakar.Severity = int(input("Enter severity risk " + str(risk.riskName)+ " for " + str(pakar.role) + ": "))
            pakar.Occurance = int(input("Enter occurance risk " + str(risk.riskName)+ " for " + str(pakar.role) + ": "))
            pakar.Detection = int(input("Enter detection risk " + str(risk.riskName)+ " for " + str(pakar.role) + ": "))

In [5]:
# Input linguistik

for kategori in listKategori:
    for key in kategori.S.keys():
        kategori.S[key] = input("Enter linguistik S " + key + " for " + kategori.kategoriName)
    for key in kategori.O.keys():
        kategori.O[key] = input("Enter linguistik O " + key + " for " + kategori.kategoriName)
    for key in kategori.D.keys():
        kategori.D[key] = input("Enter linguistik D " + key + " for " + kategori.kategoriName)

In [41]:
# Rank

df = pd.DataFrame(columns=['Kategori', 'Risk', 'S', 'O', 'D', 'FRPN'])

for kategori in listKategori:
    for risk in kategori.listResiko:
        s = round(round(risk.RIS(), 2)**round(kategori.MWS(), 3), 3)
        o = round(round(risk.RIO(), 2)**round(kategori.MWO(), 3), 3)
        d = round(round(risk.RID(), 2)**round(kategori.MWD(), 3), 3)
        frpn = round(s*o*d, 3)
        df.loc[len(df)] = [kategori.kategoriName, risk.riskName, s, o, d, frpn]

2610
1530
2610


In [50]:
df["rank"] = df.groupby("Kategori")["FRPN"].rank(method="min", ascending=False)

In [51]:
df

,Kategori,Risk,S,O,D,FRPN,rank
0,Petani,A1,0.910,0.939,0.925,0.790,1.0
1,Petani,A2,0.832,0.736,0.966,0.592,2.0
2,Roaster,B1,0.906,0.940,0.843,0.718,1.0
3,Roaster,B2,0.906,0.940,0.843,0.718,1.0
4,Roaster,B2,0.906,0.940,0.843,0.600,3.0
